In [1]:
import os
import PIL
from PIL import Image
from pathlib import Path

In [2]:
#Obtaining the dataset path and listing all image files, plus the number of images found
dataset_path = Path('../dataset')
image_paths = []
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith(valid_extensions):
            image_paths.append(os.path.join(root, file))

print(f"Total images found: {len(image_paths)}")

Total images found: 4217


In [3]:
# Exploring the dataset and checking for missing labels or broken images

broken_images = []
for image in image_paths:
    try:
        img_path = Path('../dataset') / image
        img = Image.open(img_path)
        img.verify()  # Verify that the image is not broken
        img = Image.open(img_path)
        img.load()  # Load the image to ensure it's not corrupted
    except (IOError, SyntaxError) as e:
        print('Broken image:', image)
        broken_images.append(image)
        
print('Total broken images:', len(broken_images))

Total broken images: 0


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from tensorflow.keras.preprocessing.image import load_img, img_to_array # type: ignore


In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)  # rescales automatically
train_generator = train_datagen.flow_from_directory(
    '../dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 0 images belonging to 3 classes.


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE     = (224, 224)
BATCH_SIZE   = 32

# 1. Training generator (augmentation + rescale)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

train_gen = train_datagen.flow_from_directory(
    '../dataset/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# 2. Validation generator (only rescale)
val_datagen = ImageDataGenerator(rescale=1./255)
val_gen = val_datagen.flow_from_directory(
    '../dataset/val',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# 3. Test generator (only rescale, no shuffle)
test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(
    '../dataset/test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 0 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Found 0 images belonging to 3 classes.


In [7]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense

# 1. Load pre-trained ResNet50 without top layers
base_model = ResNet50(
    weights='imagenet',  # Corrected weights name
    include_top=False,    # Corrected parameter name
    input_shape=(224, 224, 3)  # Fixed shape (height, width, channels)
)

# 2. Freeze convolutional base
for layer in base_model.layers:  # Corrected variable name
    layer.trainable = False

# 3. Add custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Corrected layer name
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)  # Added units (256 is typical)
outputs = Dense(4, activation='softmax')(x)  # 4 classes for eye conditions

model = Model(inputs=base_model.input, outputs=outputs)

# 4. Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Corrected loss function
    metrics=['accuracy']  # Corrected metrics parameter
)

In [8]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [9]:
# Create validation generator
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    '../dataset/val',  # Use your validation folder
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Verify it found images
print(f"Found {val_generator.samples} validation images")

Found 0 images belonging to 3 classes.
Found 0 validation images


In [10]:
history = model.fit(
    train_generator,
    validation_data=val_generator,  # Fixed parameter name
    epochs=10,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size
)

c:\Users\Gaming 15\Dropbox\University - Copy\Coding and Programming\Eye Disease Recess Assignment\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: The PyDataset has length 0